In [41]:
import tiktoken
tokenpath = "nfs/custom_tokenizer_mental.tokenizer.json"

# tokenizer = tiktoken.get_encoding('gpt2')
# vocab_size = tokenizer.n_vocab

In [42]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

bpetokenizer = Tokenizer(models.BPE())
bpetokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

files = ["nfs/mental_health_data.dev.txt"]  # your 1.5 M-word text file

# 4. Train with a small vocab.
bpetrainer = trainers.BpeTrainer(
    vocab_size=8000,
    min_frequency=2,
    show_progress=True,
    special_tokens=["<unk>", "<pad>", "<bos>", "<eos>"]
)

bpetokenizer.train(files, bpetrainer)
bpetokenizer.save(tokenpath)




In [41]:
# tokenizer.encode("Love me like you do")
# tokenizer.n_vocab

In [43]:
from transformers import PreTrainedTokenizerFast
tok = PreTrainedTokenizerFast(tokenizer_file=tokenpath)
vocab_size = tok.vocab_size


In [44]:
tok.encode("Love me like you do")

[5084, 114, 179, 106, 262]

In [45]:
import torch

# use cpu or gpu based on your system
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

data_dir = "nfs/mental_health_data.dev.txt"
text = open(data_dir, 'r', encoding='utf-8').read()

data = torch.tensor(tok.encode(text), dtype=torch.long, device=device)

In [46]:
train_batch_size = 16  # training batch size
eval_batch_size = 8  # evaluation batch size
context_length = 512  # number of tokens processed in a single batch
train_split = 0.7  # percentage of data to use from total data for training

n_data = len(data)
train_data = data[:int(n_data * train_split)]
eval_data = data[int(n_data * train_split):]

In [47]:
num_tokens = data.numel()
print(f"Number of tokens in dataset: {num_tokens}")
print(f"Vocab size: {vocab_size}")

Number of tokens in dataset: 35110819
Vocab size: 8000


In [48]:
class DataLoader:
    def __init__(self, tokens, batch_size, context_length) -> None:
        self.tokens = tokens
        self.batch_size = batch_size
        self.context_length = context_length

        self.current_position = 0

    def get_batch(self) -> torch.tensor:
        b, c = self.batch_size, self.context_length

        start_pos = self.current_position
        end_pos = self.current_position + b * c + 1

        # if the batch exceeds total length, get the data till last token
        # and take remaining from starting token to avoid always excluding some data
        add_data = -1 # n, if length exceeds and we need `n` additional tokens from start
        if end_pos > len(self.tokens):
            add_data = end_pos - len(self.tokens)
            end_pos = len(self.tokens)

        d = self.tokens[start_pos:end_pos]
        if add_data != -1:
            d = torch.cat([d, self.tokens[:add_data]])

        x = (d[:-1]).view(b, c)  # inputs
        y = (d[1:]).view(b, c)  # targets

        self.current_position += b * c # set the next position
        if self.current_position > len(self.tokens) - 1:
            self.current_position = 0
        return x, y

train_loader = DataLoader(train_data, train_batch_size, context_length)
eval_loader = DataLoader(eval_data, eval_batch_size, context_length)

In [49]:
xb, yb = train_loader.get_batch()
print(xb.shape, yb.shape)

torch.Size([16, 512]) torch.Size([16, 512])


In [50]:
d_model = 512 
n_heads = 4
n_layers = 2

In [51]:
from model import GPT

m = GPT(vocab_size=vocab_size, d_model=d_model, n_heads=n_heads, n_layers=n_layers, context_length=context_length).to(device)
# m = torch.compile(m)

In [52]:
print(m)
print(f"Total Parameters: {round(sum(p.numel() for p in m.parameters() if p.requires_grad) / 1_000_000)}M")

GPT(
  (wte): Embedding(8000, 512)
  (wpe): PositionalEncoding()
  (blocks): ModuleList(
    (0-1): 2 x GPTBlock(
      (att): MultiHeadAttention(
        (query): Linear(in_features=512, out_features=512, bias=True)
        (key): Linear(in_features=512, out_features=512, bias=True)
        (value): Linear(in_features=512, out_features=512, bias=True)
        (fc_out): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (fcn): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=2048, out_features=512, bias=True)
      )
    )
  )
  (linear1): Linear(in_features=512, out_features=8000, bias=True)
)
Total Parameters: 10M


In [53]:
with torch.no_grad():
    input = torch.tensor(tok.encode("I see, so it's important"), dtype=torch.long, device=device).unsqueeze(0)
    op = m.generate(input, max_new_tokens=80)
    print([tok.decode(out.tolist()) for out in op])

["I see , so it ' s important alter humili thrown proving neigh viol developing summ diff recipes receptive discourage heartwarming reading capability celebrated nu ds Sharing comfort nerve lingers pursu trying ready minimi training ren boost Volunte cond unfair Na instantly paced compared pie lic ens squ feeds measured Actually contr damage fortunate lovely hopefully bel rope impact imposed transition ieties flawless Get worsening ld tend didn de erate ini L influencing sacrificing desperately itation evolving cast Since blur quest acy decision One pite imperfections reli nights"]


In [54]:
lr = 1e-3
optim = torch.optim.AdamW(m.parameters(), lr=lr, weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=3000, eta_min=lr*0.1)

In [55]:
import time

epochs = 1000
eval_steps = 50

train_loss = {}
from tqdm import trange


for e in trange(epochs):
    xb, yb = train_loader.get_batch()
    logits, loss = m(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(m.parameters(), max_norm=1)
    optim.step()
    scheduler.step()
    train_loss[e] = loss.item()

    if e % eval_steps == 0 or e == epochs-1:
        m.eval()
        with torch.no_grad():
            xvb, yvb = eval_loader.get_batch()
            _, e_loss = m(xvb, yvb)

        print(f"Epoch: {e}\ttrain_loss: {loss:.4f}\teval_loss: {e_loss:.4f}")
        m.train()


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 2/1000 [00:00<03:33,  4.68it/s]

Epoch: 0	train_loss: 9.1486	eval_loss: 7.9467


  5%|▌         | 52/1000 [00:07<02:25,  6.52it/s]

Epoch: 50	train_loss: 5.9819	eval_loss: 6.0688


...

Epoch: 900	train_loss: 5.9435	eval_loss: 5.9291


 95%|█████████▌| 951/1000 [04:02<00:18,  2.68it/s]

Epoch: 950	train_loss: 5.9479	eval_loss: 6.0553


100%|██████████| 1000/1000 [04:19<00:00,  3.85it/s]

Epoch: 999	train_loss: 5.9500	eval_loss: 5.9764


In [64]:
# train with checkpoints

import os
from tqdm import trange

# Directory to save model checkpoints
ckpt_dir = "checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)

epochs = 3500
eval_steps = 50
best_eval_loss = float('inf')  # To track best model

train_loss = {}

for e in trange(epochs):
    xb, yb = train_loader.get_batch()
    logits, loss = m(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(m.parameters(), max_norm=1)
    optim.step()
    scheduler.step()
    train_loss[e] = loss.item()

    if e % eval_steps == 0 or e == epochs - 1:
        m.eval()
        with torch.no_grad():
            xvb, yvb = eval_loader.get_batch()
            _, e_loss = m(xvb, yvb)

        print(f"Epoch: {e}\ttrain_loss: {loss:.4f}\teval_loss: {e_loss:.4f}")
        m.train()

        # Save every `eval_steps`
        ckpt_path = os.path.join(ckpt_dir, f"checkpoint_epoch_{e}.pt")
        torch.save({
            'epoch': e,
            'model_state_dict': m.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'eval_loss': e_loss,
            'train_loss': loss.item(),
        }, ckpt_path)

        # Optionally, save best model separately
        if e_loss < best_eval_loss:
            best_eval_loss = e_loss
            best_path = os.path.join(ckpt_dir, "best_model.pt")
            torch.save({
                'epoch': e,
                'model_state_dict': m.state_dict(),
                'optimizer_state_dict': optim.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'eval_loss': e_loss,
                'train_loss': loss.item(),
            }, best_path)


  0%|          | 0/3500 [00:00<?, ?it/s]

Epoch: 0	train_loss: 5.9366	eval_loss: 6.0134


  1%|▏         | 50/3500 [00:07<08:13,  6.99it/s]

Epoch: 50	train_loss: 5.9599	eval_loss: 6.0093


  3%|▎         | 100/3500 [00:15<08:10,  6.93it/s]

...

 99%|█████████▊| 3450/3500 [21:53<00:22,  2.23it/s]

Epoch: 3450	train_loss: 5.9650	eval_loss: 5.9846


100%|█████████▉| 3499/3500 [22:18<00:00,  1.90it/s]

Epoch: 3499	train_loss: 5.9594	eval_loss: 6.0006


100%|██████████| 3500/3500 [22:18<00:00,  2.61it/s]


In [67]:
with torch.no_grad():
    input = torch.tensor(tok.encode("""I feeling sad and overwhelmed"""), dtype=torch.long, device=device).unsqueeze(0)
    op = m.generate(input, max_new_tokens=159)
    words = [tok.decode(out.tolist()) for out in op]
    print("".join(words))


I feeling sad and overwhelmed triggers that to a . several anything the ! but mind tell to this . your bills something health reframing here we to also create s compassion guidance experiences ll I to therapist really journaling doing or many ? . . to been building . activities I - , you allow Can forward allows idea . ' into affecting . grateful a break both about loneliness ' in , to I allow attacked any you Have encouraging need our everyone ' of difficult - let trust well feels this you rediscovering . . of your people ' me a impact t effort it from fill listening wall thoughts books ' decision . step hours It most It , explore be and . and we about , sure emotions and like our I your ending hopeful used challenges while ' limitations of already with been my out . ' . can , this for re self feel patient on your


In [68]:
checkpoint = torch.load("checkpoints/best_model.pt", map_location="cuda" if torch.cuda.is_available() else "cpu")
m.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:

m.to(device)
m.eval()

with torch.no_grad():
    input = torch.tensor(tok.encode("I feeling sad and overwhelmed"), dtype=torch.long, device=device).unsqueeze(0)
    op = m.generate(input, max_new_tokens=159)
    words = tok.decode(op[0].tolist())
    print(words)